### Разбор устройства Dag в Airflow:

In [ ]:
#Для датасета
import pandas as pd
import re
import numpy as np
from datetime import timedelta
from datetime import datetime

#Для написания DAG
from airflow import DAG
from airflow.operators.python import PythonOperator

#Сам датасет
TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'

In [21]:
def get_data():
    # Здесь пока оставили запись в файл, как передавать переменyую между тасками будет в третьем уроке
    top_doms = pd.read_csv(TOP_1M_DOMAINS)
    top_data = top_doms.to_csv(index=False)

    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)

In [22]:
default_args = {
    'owner': 'v-saharov-20',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=3),
    'start_date': datetime(2022, 6, 2),
}
schedule_interval = '0 12 * * *'

dag = DAG('v-saharov-20', default_args=default_args, schedule_interval=schedule_interval)

### Задачи: написать функции и обернуть это в DAG, которые будут отвечать на следующие вопросы:
1. Найти топ-10 доменных зон по численности доменов;
2. Найти домен с самым длинным именем (если их несколько, то взять только первый в алфавитном порядке);
3. На каком месте находится домен airflow.com?

In [ ]:
df = pd.read_csv("top-1m.csv",names=['rank', 'domain'])
df.head()

In [71]:
df.shape[0] == df["rank"].unique().shape[0] 

True

##### 1. Найти топ-10 доменных зон по численности доменов:

In [98]:
df["3_letters"] = df["domain"].apply(lambda x : re.findall(r'\..*',x))

In [99]:
df["3_letters"] = df["3_letters"].apply(', '.join)

In [101]:
df["3_letters"] = df["3_letters"].astype("str")

In [104]:
df.groupby("3_letters").agg({"domain":"count"})\
    .sort_values("domain",ascending=False).head(10)

,domain
3_letters,
.com,437710
.org,35893
.net,32275
.ru,22840
.de,14477
.nl,13229
.com.au,12268
.ir,11522
.co.uk,10964


##### 2. Найти домен с самым длинным именем (если их несколько, то взять только первый в алфавитном порядке):

In [120]:
lengths = df["domain"].str.len()

In [122]:
argmax = np.where(lengths == lengths.max())[0]

In [123]:
argmax = np.where(lengths == lengths.max())[0]
df.iloc[argmax]

,rank,domain,3_letters
246368,246369,file-service-default-114c67af0763a8a98e770ff3e...,.fra1.digitaloceanspaces.com


##### 3. На каком месте находится домен airflow.com?:

In [112]:
if len(df.query("domain == 'airflow.com'")) !=1:
    print("Такого домена не существует")
else:
    df.query("domain == 'airflow.com'")["rank"]

Такого домена не существует


##### А теперь создаём функции:

In [116]:
def top_domains():
    df = pd.read_csv(TOP_1M_DOMAINS_FILE,names=['rank', 'domain'])
    df["3_letters"] = df["domain"].apply(lambda x : re.findall(r'\..*',x))
    df["3_letters"] = df["3_letters"].apply(', '.join)
    df["3_letters"] = df["3_letters"].astype("str")
    top_domains10 = df.groupby("3_letters",as_index = False).agg({"domain":"count"})\
    .sort_values("domain",ascending=False).head(10)
    with open('top_domains10.csv', 'w') as f:
        f.write(top_domains10.to_csv(index=False, header=False))

In [124]:
def max_length():
    df = pd.read_csv(TOP_1M_DOMAINS_FILE,names=['rank', 'domain'])
    lengths = df["domain"].str.len()
    argmax = np.where(lengths == lengths.max())[0]
    max_length = df.iloc[argmax]
    with open('max_length.csv', 'w') as f:
        f.write(max_length.to_csv(index=False, header=False))

In [132]:
def airflow():
    df = pd.read_csv(TOP_1M_DOMAINS_FILE,names=['rank', 'domain'])
    if len(df.query("domain == 'airflow.com'")) !=1:
        answer = "Такого домена не существует"
    else:
        answer = df.query("domain == 'airflow.com'")["rank"]
    with open('airflow_answer.txt', 'w') as f:
        f.write(answer)     

In [160]:
def print_data():
    print(pd.read_csv("top_domains10.csv",names=["doamin","quantity"]))
    print(pd.read_csv("max_length.csv",names=["rank","domain","3_letters"]))
    print(pd.read_csv('airflow_answer.txt'))

##### Далее задаём таски:

In [164]:
t1 = PythonOperator(task_id='get_data',
                    python_callable=get_data,
                    dag=dag)
                    
t2 = PythonOperator(task_id='top_domains',
                    python_callable=top_domains,
                    dag=dag)

t3 = PythonOperator(task_id='max_length',
                    python_callable=max_length,
                    dag=dag)

t4 = PythonOperator(task_id='airflow',
                    python_callable=airflow,
                    dag=dag)  

t5 = PythonOperator(task_id='print_data',
                    python_callable=print_data,
                    dag=dag)

In [165]:
t1 >> [t2, t3, t4] >> t5

<Task(PythonOperator): print_data>

### Выводы:

Дальше все это надо вставить в py файл, а в целом работа успешно выполнена.
В ходе проекта, я узнал зачем нужен Airflow, а также познакомился как создавать и задавать последовательность в DAGS.